In [403]:
using CSV, DataFrames

In [404]:
root = dirname(@__FILE__)
inputAIS = joinpath(root, "AI_factor_full.txt")


"/home/lali/TITAN-ROG-sync/julia/ISI/AI_factor_full.txt"

In [405]:
#AIS = DataFrame(CSV.File(inputAIS))
AIS = CSV.read(inputAIS, DataFrame; header=["Journal","1997","1998","1999",
                                            "2000","2001","2002","2003","2004",
                                            "2005","2006","2007","2008","2009",
                                            "2010","2011","2012","2013","2014",
                                            "2015","2016","2017","2018","2019"], missingstring=["Not Available",""])
AIS = select!(AIS, Not(:"2019"));
AIS."2018" = replace(AIS."2018", missing => 0);
eltype.(eachcol(AIS));

In [407]:
root = dirname(@__FILE__)
inputISI = joinpath(root, "IF_factor_full.txt")

"/home/lali/TITAN-ROG-sync/julia/ISI/IF_factor_full.txt"

In [408]:
ISI = CSV.read(inputISI, DataFrame; header=["Journal","1997","1998","1999",
                                            "2000","2001","2002","2003","2004",
                                            "2005","2006","2007","2008","2009",
                                            "2010","2011","2012","2013","2014",
                                            "2015","2016","2017","2018","2019"], missingstring=["Not Available",""])
ISI = select!(ISI, Not(:"2019"));
ISI."2018" = replace(ISI."2018", missing => 0);
eltype.(eachcol(ISI));

In [409]:
AIS[2, [1, 2, 20, 21, 22, 23]]

,Journal,1997,2015,2016,2017,2018
,String31,Float64,Float64,Float64,Float64,Float64
2,NAT GENET,13.779,16.807,16.456,16.652,16.807


In [410]:
ISI[2, [1, 2, 20, 21, 22, 23]]

,Journal,1997,2015,2016,2017,2018
,String31,Float64,Float64,Float64,Float64,Float64
2,NAT GENET,38.854,31.616,27.959,27.125,25.455


In [411]:
root = dirname(@__FILE__)
inputAIS = joinpath(root, "ISI_Journal_Citation_Report_2019.csv")
AIS2019 = CSV.read(inputAIS, DataFrame; header=true, missingstring=["Not Available",""])
AIS2019."Article Influence Score" = replace(AIS2019."Article Influence Score", missing => 0)
AIS2019."Journal Impact Factor" = replace(AIS2019."Journal Impact Factor", missing => 0)
eltype.(eachcol(AIS2019))
#AIS2019[9567, [3, 6, 15]]
ISI2019 = AIS2019[!, [3, 6]]
AIS2019 = AIS2019[!, [3, 15]]
rename!(AIS2019,:"JCR Abbreviated Title" => :"Journal")
#rename!(AIS2019,:"Journal Impact Factor" => :"ISI2019")
rename!(AIS2019,:"Article Influence Score" => :"2019")
AIS2019[9567, :]

,Journal,2019
,String31,Float64
9567,NAT GENET,16.922


In [412]:
rename!(ISI2019,:"JCR Abbreviated Title" => :"Journal")
rename!(ISI2019,:"Journal Impact Factor" => :"2019")
#rename!(ISI2019,:"Article Influence Score" => :"2019")
ISI2019[9567, :]

,Journal,2019
,String31,Float64
9567,NAT GENET,27.603


In [413]:
root = dirname(@__FILE__)
inputAIS = joinpath(root, "ISI_Journal_Citation_Report_2020.csv")
AIS2020 = CSV.read(inputAIS, DataFrame; header=true, missingstring=["Not Available",""])
AIS2020."Article Influence Score" = replace(AIS2020."Article Influence Score", missing => 0)
AIS2020."Journal Impact Factor" = replace(AIS2020."Journal Impact Factor", missing => 0)
eltype.(eachcol(AIS2020))
#AIS2020[9685, [2, 5, 13]]
ISI2020 = AIS2020[!, [2, 5]]
AIS2020 = AIS2020[!, [2, 13]]
rename!(AIS2020,:"JCR Abbreviated Title" => :"Journal")
#rename!(AIS2020,:"Journal Impact Factor" => :"ISI2020")
rename!(AIS2020,:"Article Influence Score" => :"2020")
AIS2020[9685, :]

,Journal,2020
,String31,Float64
9685,NAT GENET,19.017


In [414]:
rename!(ISI2020,:"JCR Abbreviated Title" => :"Journal")
rename!(ISI2020,:"Journal Impact Factor" => :"2020")
#rename!(ISI2019,:"Article Influence Score" => :"2019")
ISI2020[9685, :]

,Journal,2020
,String31,Float64
9685,NAT GENET,38.33


In [415]:
AIS = leftjoin(AIS, AIS2019; on=:"Journal", makeunique=true)
AIS = leftjoin(AIS, AIS2020; on=:"Journal", makeunique=true)
AIS."2019" = replace(AIS."2019", missing => 0)
AIS."2020" = replace(AIS."2020", missing => 0)
AIS[2, [1, 2, 22, 23, 24, 25]]

,Journal,1997,2017,2018,2019,2020
,String31,Float64,Float64,Float64,Float64,Float64
2,NAT GENET,13.779,16.652,16.807,16.922,19.017


In [416]:
ISI = leftjoin(ISI, ISI2019; on=:"Journal", makeunique=true)
ISI = leftjoin(ISI, ISI2020; on=:"Journal", makeunique=true)
ISI."2019" = replace(ISI."2019", missing => 0)
ISI."2020" = replace(ISI."2020", missing => 0)
ISI[2, [1, 2, 22, 23, 24, 25]]

,Journal,1997,2017,2018,2019,2020
,String31,Float64,Float64,Float64,Float64,Float64
2,NAT GENET,38.854,27.125,25.455,27.603,38.33
